In [23]:
### testing agents

#### STUPID PYDANTIC

In [20]:
from mb_rag.agents.run_agent import AgentRunner, AgentConfig
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.tools import WolframAlphaQueryRun
import os

In [3]:
basic_agent = AgentRunner()

In [4]:
search_tool = DuckDuckGoSearchRun()
basic_agent.tools = [search_tool]

In [5]:
response = basic_agent.run_agent_chain("What is Python programming language?")
print("Basic Agent Response:", response)

ValueError: Error creating agent: This function requires a .bind_tools method be implemented on the LLM.

In [11]:
custom_config = AgentConfig(
    model_name="gpt-4",
    model_type="openai",
    prompt_type="basic",
    verbose=True
)

In [12]:
custom_agent = AgentRunner(config=custom_config)


In [21]:
wiki_tool = WikipediaQueryRun()
custom_agent.tools = [search_tool, wiki_tool]

# Run a more complex query
response = custom_agent.run_agent_chain(
    "Who was Alan Turing and what was his contribution to computer science?"
)
print("\nCustom Agent Response:", response)


ValidationError: 1 validation error for WikipediaQueryRun
api_wrapper
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [14]:
step_agent = AgentRunner.from_model(
    model_name="gpt-4",
    model_type="openai",
    verbose=True
)

In [15]:
step_agent.tools = [search_tool]

In [16]:
agent = step_agent.create_agent()

# Create the executor
executor = step_agent.create_executor(agent)

# Run a query
response = step_agent.invoke_agent(
    executor,
    "What are the fundamental principles of machine learning?"
)
print("\nStep-by-Step Agent Response:", response)


ValueError: Error creating agent: This function requires a .bind_tools method be implemented on the LLM.

In [8]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool
from typing import Type

class SimpleSearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class SimpleSearchTool(BaseTool):
    
    args_schema: Type[BaseModel] = SimpleSearchInput
        

    def _run(
        self,
        query: str,
    ) -> str:
        """Use the tool."""
        from tavily import TavilyClient

        api_key = os.getenv("TAVILY_API_KEY")
        client = TavilyClient(api_key=api_key)
        results = client.search(query=query)
        return f"Search results for: {query}\n\n\n{results}\n"


In [10]:
tools = [SimpleSearchTool()]

ValidationError: 2 validation errors for SimpleSearchTool
name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
description
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [17]:
queries = [
    "What is quantum computing?",
    "How does artificial intelligence work?",
    "What are neural networks?"
]

batch_agent = AgentRunner.from_model(
    model_name="gpt-4",
    model_type="openai"
)
batch_agent.tools = [search_tool, wiki_tool]

print("\nBatch Processing Results:")
for query in queries:
    print(f"\nQuery: {query}")
    response = batch_agent.run_agent_chain(query)
    print(f"Response: {response}")



NameError: name 'wiki_tool' is not defined

In [18]:
advanced_config = AgentConfig(
    model_name="gpt-4",
    model_type="openai",
    prompt_type="basic",
    verbose=True,
    handle_parsing_errors=True
)

advanced_agent = AgentRunner(advanced_config)
advanced_agent.tools = [search_tool, wiki_tool]

agent = advanced_agent.create_agent()
executor = advanced_agent.create_executor(
    agent,
    max_iterations=3,
    early_stopping_method="generate"
)

response = advanced_agent.invoke_agent(
    executor,
    "Explain the concept of quantum entanglement and its applications"
)
print("\nAdvanced Agent Response:", response)

NameError: name 'wiki_tool' is not defined